In [18]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [19]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [20]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [21]:
import pandas as pd
import numpy as np
import xgboost as xgb
import torch
import  matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
import holidays
from sklearn.model_selection import GridSearchCV


In [22]:
# GPU or CPU use for model
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

In [23]:
PATH = '/content/drive/My Drive/Enefit/'

In [24]:
train_df = pd.read_csv("train.csv")
gas_df = pd.read_csv("gas_prices.csv")
electricity_df = pd.read_csv("electricity_prices.csv")
client_df = pd.read_csv("client.csv")
fw_df = pd.read_csv("forecast_weather.csv")
hw_df = pd.read_csv("historical_weather.csv")
location = pd.read_csv("county_lon_lats.csv")
location = location.drop(columns = ["Unnamed: 0"])

In [25]:
class FeatureProcessorClass():
    def __init__(self):
        # Columns to join on for the different datasets
        self.weather_join = ['datetime', 'county', 'data_block_id']
        self.gas_join = ['datetime','data_block_id']
        self.electricity_join = ['datetime', 'data_block_id']
        self.client_join = ['county', 'is_business', 'product_type', 'data_block_id']
        self.holiday = ['datetime']
        # Columns of latitude & longitude
        self.lat_lon_columns = ['latitude', 'longitude']

        # Aggregate stats
        self.agg_stats = ['mean'] #, 'min', 'max', 'std', 'median']

        # Categorical columns (specify for XGBoost)
        self.category_columns = ['county', 'is_business', 'product_type', 'is_consumption', 'data_block_id','holiday']

    def create_new_column_names(self, df, suffix, columns_no_change):
        '''Change column names by given suffix, keep columns_no_change, and return back the data'''
        df.columns = [col + suffix
                      if col not in columns_no_change
                      else col
                      for col in df.columns
                      ]
        return df

    def flatten_multi_index_columns(self, df):
        df.columns = ['_'.join([col for col in multi_col if len(col)>0])
                      for multi_col in df.columns]
        return df

    def create_data_features(self, data):
        '''📊Create features for main data (test or train) set📊'''
        # To datetime
        data['datetime'] = pd.to_datetime(data['datetime'])

        # Time period features
        data['date'] = data['datetime'].dt.normalize()
        data['year'] = data['datetime'].dt.year
        data['quarter'] = data['datetime'].dt.quarter
        data['month'] = data['datetime'].dt.month
        data['week'] = data['datetime'].dt.isocalendar().week
        data['hour'] = data['datetime'].dt.hour

        # Day features
        data['day_of_year'] = data['datetime'].dt.day_of_year
        data['day_of_month']  = data['datetime'].dt.day
        data['day_of_week'] = data['datetime'].dt.day_of_week
        return data

    def create_client_features(self, client):
        '''💼 Create client features 💼'''
        # Modify column names - specify suffix
        client = self.create_new_column_names(client,
                                           suffix='_client',
                                           columns_no_change = self.client_join
                                          )
        client['data_block_id']-=2
        return client

    def create_historical_weather_features(self, historical_weather):
        '''⌛🌤️ Create historical weather features 🌤️⌛'''

        # To datetime
        historical_weather['datetime'] = pd.to_datetime(historical_weather['datetime'])

        # Add county
        historical_weather[self.lat_lon_columns] = historical_weather[self.lat_lon_columns].astype(float).round(1)
        historical_weather = historical_weather.merge(location, how = 'left', on = self.lat_lon_columns)

        # Modify column names - specify suffix
        historical_weather = self.create_new_column_names(historical_weather,
                                                          suffix='_h',
                                                          columns_no_change = self.lat_lon_columns + self.weather_join
                                                          )

        # Group by & calculate aggregate stats
        agg_columns = [col for col in historical_weather.columns if col not in self.lat_lon_columns + self.weather_join]
        agg_dict = {agg_col: self.agg_stats for agg_col in agg_columns}
        historical_weather = historical_weather.groupby(self.weather_join).agg(agg_dict).reset_index()
        # Flatten the multi column aggregates
        historical_weather = self.flatten_multi_index_columns(historical_weather)

        # Test set has 1 day offset for hour<11 and 2 day offset for hour>11
        # historical_weather['hour_h'] = historical_weather['datetime'].dt.hour
        # historical_weather['datetime'] = (historical_weather
        #                                        .apply(lambda x:
        #                                               x['datetime'] + pd.DateOffset(1)
        #                                               if x['hour_h']< 11
        #                                               else x['datetime'] + pd.DateOffset(2),
        #                                               axis=1)
        #                                       )
        historical_weather['data_block_id'] = historical_weather['data_block_id'].astype(int)
        historical_weather['data_block_id'] -= 1

        return historical_weather

    def create_forecast_weather_features(self, forecast_weather):
        '''🔮🌤️ Create forecast weather features 🌤️🔮'''

        # Rename column and drop
        forecast_weather = (forecast_weather
                            .rename(columns = {'forecast_datetime': 'datetime'})
                            .drop(columns = 'origin_datetime')
                           )

        # To datetime
        forecast_weather['datetime'] = (pd.to_datetime(forecast_weather['datetime'])
                                        .dt
                                        .tz_localize(None)
                                       )

        # Add county
        forecast_weather[self.lat_lon_columns] = forecast_weather[self.lat_lon_columns].astype(float).round(1)
        forecast_weather = forecast_weather.merge(location, how = 'left', on = self.lat_lon_columns)

        # Modify column names - specify suffix
        forecast_weather = self.create_new_column_names(forecast_weather,
                                                        suffix='_f',
                                                        columns_no_change = self.lat_lon_columns + self.weather_join
                                                        )

        # Group by & calculate aggregate stats
        agg_columns = [col for col in forecast_weather.columns if col not in self.lat_lon_columns + self.weather_join]
        agg_dict = {agg_col: self.agg_stats for agg_col in agg_columns}
        forecast_weather = forecast_weather.groupby(self.weather_join).agg(agg_dict).reset_index()

        # Flatten the multi column aggregates
        forecast_weather = self.flatten_multi_index_columns(forecast_weather)
        forecast_weather['data_block_id'] -= 1
        return forecast_weather

    def create_electricity_features(self, electricity):
        '''⚡ Create electricity prices features ⚡'''
        # To datetime
        electricity['forecast_date'] = pd.to_datetime(electricity['forecast_date'])

        # Test set has 1 day offset
        electricity['datetime'] = electricity['forecast_date']

        # Modify column names - specify suffix
        electricity = self.create_new_column_names(electricity,
                                                   suffix='_electricity',
                                                   columns_no_change = self.electricity_join)
        electricity['data_block_id']-=1
        return electricity

    def create_gas_features(self, gas):
        '''⛽ Create gas prices features ⛽'''
        gas['forecast_date'] = pd.to_datetime(gas['forecast_date'])
        gas['datetime'] = gas['forecast_date']

        # Mean gas price
        gas['mean_price_per_mwh'] = (gas['lowest_price_per_mwh'] + gas['highest_price_per_mwh'])/2

        # Modify column names - specify suffix
        gas = self.create_new_column_names(gas,
                                           suffix='_gas',
                                           columns_no_change = self.gas_join
                                          )
        gas['data_block_id']-=1

        return gas

    def get_holiday_features(self, df, country_code='US'):

        year_range = list(range(min(df['datetime'].dt.year), max(df['datetime'].dt.year) + 1))
        country_holidays = holidays.country_holidays(
        country_code,
        years=year_range,
        observed=False
        )
        holiday = pd.DataFrame(country_holidays.items())
        holiday.columns = ['date', 'holiday']
        holiday['date'] = pd.to_datetime(holiday['date'])
        holiday = holiday.rename(columns={'date': 'datetime'})
        holiday['datetime'] = pd.to_datetime(holiday['datetime'])

        return holiday
    def __call__(self, data, client, historical_weather, forecast_weather, electricity, gas):
        '''Processing of features from all datasets, merge together and return features for dataframe df '''
        # Create features for relevant dataset
        data = self.create_data_features(data)
        client = self.create_client_features(client)
        historical_weather = self.create_historical_weather_features(historical_weather)
        forecast_weather = self.create_forecast_weather_features(forecast_weather)
        electricity = self.create_electricity_features(electricity)
        gas = self.create_gas_features(gas)
        holiday = self.get_holiday_features(data)
        # 🔗 Merge all datasets into one df 🔗
        df = data.merge(client, how='left', on = self.client_join)
        df = df.merge(historical_weather, how='left', on = self.weather_join)
        df = df.merge(forecast_weather, how='left', on = self.weather_join)
        df = df.merge(electricity, how='left', on = self.electricity_join)
        df = df.merge(gas, how='left', on = self.gas_join)
        df = df.merge(holiday, how='left', on = self.holiday)
        # Assuming 'df' is your DataFrame containing the 'holiday' column
        df['holiday'] = df['holiday'].fillna(0)  # Fill NaN values with 0
        df.loc[df['holiday'] != 0, 'holiday'] = 1  # Change non-zero values to 1


        # Change columns to categorical for XGBoost
        df[self.category_columns] = df[self.category_columns].astype('category')
        return df

In [26]:
def create_revealed_targets_train(data, N_day_lags):
    '''🎯 Create past revealed_targets for train set based on number of day lags N_day_lags 🎯 '''
    original_datetime = data['datetime']
    revealed_targets = data[['datetime', 'prediction_unit_id', 'is_consumption', 'target']].copy()

    # Create revealed targets for all day lags
    for day_lag in range(2, N_day_lags+1):
        revealed_targets['datetime'] = original_datetime + pd.DateOffset(day_lag)
        data = data.merge(revealed_targets,
                          how='left',
                          on = ['datetime', 'prediction_unit_id', 'is_consumption'],
                          suffixes = ('', f'_{day_lag}_days_ago')
                         )
    return data


In [27]:
%%time
# Create all features

FeatureProcessor = FeatureProcessorClass()

data = FeatureProcessor(data = train_df.copy(),
                      client = client_df.copy(),
                      historical_weather = hw_df.copy(),
                      forecast_weather = fw_df.copy(),
                      electricity = electricity_df.copy(),
                      gas = gas_df.copy(),
                     )


CPU times: user 5.26 s, sys: 3.72 s, total: 8.98 s
Wall time: 9.89 s


In [28]:
data.head()

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id,date,...,total_precipitation_f_mean,forecast_date_electricity,euros_per_mwh_electricity,origin_date_electricity,forecast_date_gas,lowest_price_per_mwh_gas,highest_price_per_mwh_gas,origin_date_gas,mean_price_per_mwh_gas,holiday
0,0,0,1,0.713,0,2021-09-01,0,0,0,2021-09-01,...,NaN,2021-09-01,92.51,2021-08-31 00:00:00,2021-09-01,45.23,46.32,2021-08-31,45.775,0
1,0,0,1,96.590,1,2021-09-01,0,1,0,2021-09-01,...,NaN,2021-09-01,92.51,2021-08-31 00:00:00,2021-09-01,45.23,46.32,2021-08-31,45.775,0
2,0,0,2,0.000,0,2021-09-01,0,2,1,2021-09-01,...,NaN,2021-09-01,92.51,2021-08-31 00:00:00,2021-09-01,45.23,46.32,2021-08-31,45.775,0
3,0,0,2,17.314,1,2021-09-01,0,3,1,2021-09-01,...,NaN,2021-09-01,92.51,2021-08-31 00:00:00,2021-09-01,45.23,46.32,2021-08-31,45.775,0
4,0,0,3,2.904,0,2021-09-01,0,4,2,2021-09-01,...,NaN,2021-09-01,92.51,2021-08-31 00:00:00,2021-09-01,45.23,46.32,2021-08-31,45.775,0


In [29]:
N_day_lags = 7 # Specify how many days we want to go back (at least 2)

df = create_revealed_targets_train(data.copy(),
                                  N_day_lags = N_day_lags)

In [ ]:
df['mean_target_days'] = df[[f"target_{i}_days_ago" for i in range(2, 15)]].mean(axis=1)

In [30]:
# Remove empty target row
target = 'target'
df = df[df[target].notnull()].reset_index(drop=True)

In [31]:
#### Create single fold split ######
train_block_id = list(range(0, 600))

tr = df[df['data_block_id'].isin(train_block_id)] # first 600 data_block_ids used for training
val = df[~df['data_block_id'].isin(train_block_id)] # rest data_block_ids used for validation

In [32]:
# Remove columns for features
no_features = ['date',
                'latitude',
                'longitude',
                'data_block_id',
                'row_id',
                'hours_ahead',
                'hour_h',
               ]

remove_columns = [col for col in df.columns for no_feature in no_features if no_feature in col]
remove_columns


['datetime',
 'data_block_id',
 'row_id',
 'date',
 'date_client',
 'hours_ahead_f_mean',
 'forecast_date_electricity',
 'origin_date_electricity',
 'forecast_date_gas',
 'origin_date_gas']

In [33]:
remove_columns.append(target)
remove_columns

['datetime',
 'data_block_id',
 'row_id',
 'date',
 'date_client',
 'hours_ahead_f_mean',
 'forecast_date_electricity',
 'origin_date_electricity',
 'forecast_date_gas',
 'origin_date_gas',
 'target']

In [34]:
features = [col for col in df.columns if col not in remove_columns]
features

['county',
 'is_business',
 'product_type',
 'is_consumption',
 'prediction_unit_id',
 'year',
 'quarter',
 'month',
 'week',
 'hour',
 'day_of_year',
 'day_of_month',
 'day_of_week',
 'eic_count_client',
 'installed_capacity_client',
 'temperature_h_mean',
 'dewpoint_h_mean',
 'rain_h_mean',
 'snowfall_h_mean',
 'surface_pressure_h_mean',
 'cloudcover_total_h_mean',
 'cloudcover_low_h_mean',
 'cloudcover_mid_h_mean',
 'cloudcover_high_h_mean',
 'windspeed_10m_h_mean',
 'winddirection_10m_h_mean',
 'shortwave_radiation_h_mean',
 'direct_solar_radiation_h_mean',
 'diffuse_radiation_h_mean',
 'temperature_f_mean',
 'dewpoint_f_mean',
 'cloudcover_high_f_mean',
 'cloudcover_low_f_mean',
 'cloudcover_mid_f_mean',
 'cloudcover_total_f_mean',
 '10_metre_u_wind_component_f_mean',
 '10_metre_v_wind_component_f_mean',
 'direct_solar_radiation_f_mean',
 'surface_solar_radiation_downwards_f_mean',
 'snowfall_f_mean',
 'total_precipitation_f_mean',
 'euros_per_mwh_electricity',
 'lowest_price_per_

In [35]:
clf = xgb.XGBRegressor(
                        device = device,
                        booster='dart',
                        enable_categorical=True,
                        objective = 'reg:absoluteerror',
                        n_estimators = 1000,
                        early_stopping_rounds=100
                       )

In [ ]:
clf.fit(X = tr[features],
        y = tr[target],
        eval_set = [(tr[features], tr[target]), (val[features], val[target])],
        verbose=True
       )

[0]	validation_0-mae:241.45887	validation_1-mae:313.88049
[1]	validation_0-mae:215.81322	validation_1-mae:282.69834
[2]	validation_0-mae:190.54369	validation_1-mae:251.50425
[3]	validation_0-mae:169.09840	validation_1-mae:225.66084
[4]	validation_0-mae:152.37100	validation_1-mae:203.15422
[5]	validation_0-mae:138.60647	validation_1-mae:183.19125
[6]	validation_0-mae:127.65798	validation_1-mae:169.82069
[7]	validation_0-mae:107.41947	validation_1-mae:147.55239
[8]	validation_0-mae:91.04654	validation_1-mae:128.89447
[9]	validation_0-mae:79.77534	validation_1-mae:116.70160
[10]	validation_0-mae:72.33868	validation_1-mae:108.92734
[11]	validation_0-mae:66.66078	validation_1-mae:103.34935
[12]	validation_0-mae:62.48779	validation_1-mae:100.46281
[13]	validation_0-mae:59.68738	validation_1-mae:97.88880
[14]	validation_0-mae:58.08449	validation_1-mae:96.81214
[15]	validation_0-mae:57.28157	validation_1-mae:96.10353
[16]	validation_0-mae:56.27682	validation_1-mae:95.80043
[17]	validation_0-ma

In [ ]:
best_params = {
    'n_estimators': 5000,
    'verbosity': 0,  # 0 (silent), 1 (warning), 2 (info), 3 (debug)
    'objective': 'reg:tweedie',
    'num_leaves': 455,
    'learning_rate': 0.0095,
    'colsample_bytree': 0.92,
    'subsample': 0.45,
    'reg_alpha': 3.62,
    'reg_lambda': 1.65,
    'min_child_weight': 198,
    'max_depth': 21,
    'random_state': 73
}

clf_1 = XGBRegressor(**best_params)


In [ ]:
clf_1.fit(X = tr[features],
        y = tr[target],
        eval_set = [(tr[features], tr[target]), (val[features], val[target])],
        eval_metric='mae',
        early_stopping_rounds=100,
        verbose=True
       )

In [ ]:
# Plot RMSE
results = clf.evals_result()
train_mae, val_mae = results["validation_0"]["mae"], results["validation_1"]["mae"]
x_values = range(0, len(train_mae))
fig, ax = plt.subplots(figsize=(8,4))
ax.plot(x_values, train_mae, label="Train MAE")
ax.plot(x_values, val_mae, label="Validation MAE")
ax.legend()
plt.ylabel("MAE Loss")
plt.title("XGBoost MAE Loss")
plt.show()

In [ ]:
TOP = 20
importance_data = pd.DataFrame({'name': clf.feature_names_in_, 'importance': clf.feature_importances_})
importance_data = importance_data.sort_values(by='importance', ascending=False)

fig, ax = plt.subplots(figsize=(8,4))
sns.barplot(data=importance_data[:TOP],
            x = 'importance',
            y = 'name'
        )
patches = ax.patches
count = 0
for patch in patches:
    height = patch.get_height()
    width = patch.get_width()
    perc = 100*importance_data['importance'].iloc[count]#100*width/len(importance_data)
    ax.text(width, patch.get_y() + height/2, f'{perc:.1f}%')
    count+=1

plt.title(f'The top {TOP} features sorted by importance')
plt.show()